# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
%%info



In [2]:
sc.list_packages()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1589627906482_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.9.0  
boto                       2.49.0 
jmespath                   0.9.5  
lxml                       4.5.0  
mysqlclient                1.4.2  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.16.5 
pip                        9.0.1  
py-dateutil                2.2    
python37-sagemaker-pyspark 1.3.0  
pytz                       2019.3 
PyYAML                     5.3.1  
setuptools                 28.8.0 
six                        1.13.0 
soupsieve                  1.9.5  
wheel                      0.29.0 
windmill                   1.6

In [2]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository
sc.install_pypi_package("httpagentparser")
sc.install_pypi_package("seaborn")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1589646357143_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



  Running setup.py bdist_wheel for httpagentparser: started
  Running setup.py bdist_wheel for httpagentparser: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/87/aa/d6/3b45a743bf52f273cbc0efea22ebcd9347d9360eaefa4b8d49
Successfully built httpagentparser


In [41]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import desc,asc,col,sum as Fsum,udf


from pyspark.ml.feature import VectorAssembler,StringIndexer,StandardScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,GBTClassifier,NaiveBayes,RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


import matplotlib.dates as mdates
import datetime
import httpagentparser


import numpy as np
import pandas as pd
# %matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# create a Spark session

# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [5]:
# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)
# df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

In [11]:
print('number of dataset with null artist : ' ,df.where(col("artist").isNull()).count())
print('number of dataset with null userid : ' ,df.where(col("userId").isNull()).count())
print('number of dataset with null sessionid : ' ,df.where(col("sessionId").isNull()).count())
print('number of dataset with empty userid : ' ,df.where(col("userId") == '').count())
print('number of dataset with empty sessionid : ' ,df.where(col("sessionid") == '').count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

number of dataset with null artist :  5408927
number of dataset with null userid :  0
number of dataset with null sessionid :  0
number of dataset with empty userid :  0
number of dataset with empty sessionid :  0

In [6]:
#filter empty string
df_cleaned = df.filter(df.userId != '')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#Check duplicates
print("all data : ",df_cleaned.count())
print("non duplicate data : ", df_cleaned.drop_duplicates().count())
if df_cleaned.count()==df_cleaned.drop_duplicates().count():
    print('No duplicate')
else:
    print('duplicate dataset')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

all data :  26259199
non duplicate data :  26259199
No duplicate

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [16]:
#Distribution of gender
df_cleaned.groupby('gender').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+
|gender|   count|
+------+--------+
|  null|  778479|
|     M|13299562|
|     F|12181158|
+------+--------+

In [17]:
#Statistical result of song length
df_cleaned.select('length').describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|           length|
+-------+-----------------+
|  count|         20850272|
|   mean|248.7254329674883|
| stddev|97.28710387078065|
|    min|            0.522|
|    max|       3024.66567|
+-------+-----------------+

In [ ]:
df_cleaned.groupby('page').count().sort('count').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|                page|  count|
+--------------------+-------+
| Submit Registration|    401|
|            Register|    802|
|              Cancel|   5003|
|Cancellation Conf...|   5003|
|    Submit Downgrade|   6494|
|      Submit Upgrade|  15135|
|               Error|  25962|
|       Save Settings|  29516|
|             Upgrade|  50507|
|               About|  92759|
|            Settings| 147074|
|                Help| 155100|
|           Downgrade| 184240|
|         Thumbs Down| 239212|
|              Logout| 296005|
|               Login| 296350|
|          Add Friend| 381664|
|         Roll Advert| 385212|
|     Add to Playlist| 597921|
|           Thumbs Up|1151465|
+--------------------+-------+
only showing top 20 rows

In [ ]:
df_cleaned.groupby('status').count().sort('count').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+
|status|   count|
+------+--------+
|   404|   25962|
|   307| 2421245|
|   200|23811992|
+------+--------+

In [9]:
df_cleaned.createOrReplaceTempView("event")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
spark.sql("""SELECT gender, COUNT(DISTINCT userId) 
             FROM event 
             WHERE page = 'Cancellation Confirmation' GROUP BY 1 
          """).show(200)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------------+
|gender|count(DISTINCT userId)|
+------+----------------------+
|     M|                  2656|
|     F|                  2347|
+------+----------------------+

Total 5003 users is churning

In [ ]:
spark.sql("""SELECT page, COUNT(*) FROM event WHERE userId in (
             SELECT userId 
             FROM event 
             WHERE page = 'Cancellation Confirmation' LIMIT 1) GROUP BY 1 
          """).show(200)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|                page|count(1)|
+--------------------+--------+
|       Save Settings|       1|
|     Add to Playlist|       2|
|                Help|       1|
|              Cancel|       1|
|         Thumbs Down|       1|
|                Home|      11|
|         Roll Advert|       6|
|             Upgrade|       3|
|Cancellation Conf...|       1|
|            NextSong|     123|
|           Thumbs Up|       7|
|              Logout|       3|
|            Settings|       1|
+--------------------+--------+

Example of list of page that churn user visited. If you see on the 'NextSong' page, this user played 123 songs.

In [10]:
#Create new view with churn columns
event_with_churn = spark.sql("""
SELECT e.*, COALESCE(churn,0) AS churn
FROM event e
LEFT JOIN(
        SELECT userid,1 AS churn
         FROM event 
         WHERE page = 'Cancellation Confirmation'
         GROUP BY 1
         ) USING (userid)
          """)
event_with_churn.createOrReplaceTempView("event_with_churn")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark.sql(""" SELECT churn, COUNT(DISTINCT userId) FROM
event_with_churn
GROUP BY 1
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------------+
|churn|count(DISTINCT userId)|
+-----+----------------------+
|    1|                  5003|
|    0|                 17275|
+-----+----------------------+

In [ ]:
spark.sql(""" 
SELECT churn, 
       AVG(cnt_song) avg_song_played,
       AVG(cnt_song/cnt_day_played) avg_song_per_day_played,
       percentile_approx(cnt_song, 0.5) median_song_played,
       AVG(cnt_unique_song)  avg_unique_song_played,
       AVG(cnt_song)/AVG(cnt_unique_song) AS unique_ratio,
       AVG(ts-registration)/(1000*3600*365*24) AS user_age
 FROM
(SELECT userId, 
        MAX(churn) AS churn, 
        SUM(CASE WHEN page = 'NextSong' THEN 1 ELSE 0 END) cnt_song,
        COUNT(DISTINCT CASE WHEN page = 'NextSong' THEN song END) cnt_unique_song,
        MAX(ts) ts,
        COUNT(DISTINCT CASE WHEN page = 'NextSong' THEN from_unixtime(ts/1000, 'yyyy-MM-dd') ELSE 0 END) cnt_day_played,
        MAX(registration) registration
   FROM event_with_churn
 GROUP BY 1) GROUP BY 1 
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----------------+-----------------------+------------------+----------------------+------------------+------------------+
|churn|  avg_song_played|avg_song_per_day_played|median_song_played|avg_unique_song_played|      unique_ratio|          user_age|
+-----+-----------------+-----------------------+------------------+----------------------+------------------+------------------+
|    1|876.7291625024985|        58.727684128275|               512|     779.5772536478113| 1.124621271849701|3.7419776765812545|
|    0|953.0533140376266|      53.24271634785819|               563|     842.7589001447178|1.1308730336445798| 5.231680025687944|
+-----+-----------------+-----------------------+------------------+----------------------+------------------+------------------+

On average, churn user played less song compared to active user. They also have less in user_age (3.7 years compare to 5.2 years). 

In [ ]:
spark.sql("SELECT level, COUNT(*) FROM event_with_churn GROUP BY 1").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+
|level|count(1)|
+-----+--------+
| free| 5663371|
| paid|20595828|
+-----+--------+

In [ ]:
df_song_perday = spark.sql(""" 
SELECT churn, 
       cnt_song/cnt_day_played num_song_perday
 FROM
(SELECT userId, 
        MAX(churn) AS churn, 
        SUM(CASE WHEN page = 'NextSong' THEN 1 ELSE 0 END) cnt_song,
        MAX(ts) ts,
        COUNT(DISTINCT CASE WHEN page = 'NextSong' THEN from_unixtime(ts/1000, 'yyyy-MM-dd') ELSE 0 END) cnt_day_played,
        MAX(registration) registration
   FROM event_with_churn
 GROUP BY 1) 
""").toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_song_perday[df_song_perday.churn == 1].num_song_perday.hist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_song_perday[df_song_perday.churn == 0].num_song_perday.hist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Churn user has right skewed distribution in terms of song per day, which means they play less song per day. On the other hand, active users is more similar to normal distribution.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [28]:
#Create dummy column
def create_dummy_columns(col_name,df):
    '''
    Input :
    - col_name = name of the column that is used to generate dummy variables
    - df = spark df
    Output :
    - ndf = new spark df with all dummy variables from selected column
    '''
    elements = df.select(col_name).distinct().rdd.flatMap(lambda x: x).collect()
    exprs = [F.when(F.col(col_name) == element, 1).otherwise(0).alias(element)
         for element in elements]
    df_with_dummy = df.select("*", *exprs)
    
    #drop original column
    ndf = df_with_dummy.drop(col_name)
    return ndf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_with_dummy_pages = create_dummy_columns('page',event_with_churn)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def get_browser(x):
    try :
        r = httpagentparser.detect(x)['browser']['name']
    except:
        r = ''
    return r
    
def get_platform(x):
    try :
        r = httpagentparser.detect(x)['platform']['name']
    except:
        r = ''
    return r

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#browser
spark.udf.register("get_browser", get_browser)

#Platform
spark.udf.register("get_platform", get_platform)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function get_platform at 0x7f719c091dd0>

In [ ]:
#Check unique combination
spark.sql("SELECT get_browser(userAgent), get_platform(userAgent) FROM event_with_churn GROUP BY 1,2").show()

In [13]:
#Crafting session level feature
session_feature = spark.sql("""
SELECT userId, 
       AVG(hour_per_session)             AS avg_hour_per_session,
       AVG(unique_song_per_session)      AS avg_unique_song_per_session,
       AVG(unique_artist_per_session)    AS avg_unique_artist_per_session,
       AVG(cnt_song_played_persession)   AS avg_song_played_persession
FROM (       
SELECT userId,
       sessionId,
       (MAX(ts) -MIN(ts))/(1000*3600) AS hour_per_session,
       COUNT(DISTINCT song) unique_song_per_session,
       COUNT(DISTINCT artist) unique_artist_per_session,
       SUM(CASE WHEN page = 'NextSong' THEN 1 ELSE 0 END) AS cnt_song_played_persession
FROM event_with_churn 
GROUP BY 1,2)
GROUP BY 1
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
session_feature.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+---------------------------+-----------------------------+--------------------------+
| userId|avg_hour_per_session|avg_unique_song_per_session|avg_unique_artist_per_session|avg_song_played_persession|
+-------+--------------------+---------------------------+-----------------------------+--------------------------+
|1338783|  7.9227951388888895|                     101.75|                      95.8125|                  104.5625|
|1271218|   6.910691721132898|          91.76470588235294|            87.07843137254902|         93.70588235294117|
|1396135|   7.612685185185185|         105.88888888888889|                        101.0|        108.22222222222223|
|1690101|   4.364001501501501|         63.513513513513516|            61.37837837837838|         64.37837837837837|
|1676292|  10.208055555555553|         144.72727272727272|           132.45454545454547|         149.0909090909091|
|1380035|  5.4878806584362145|          71.25925925925925|            69

In [14]:
session_feature.createOrReplaceTempView('session_feature')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#Crafting the feature
full_feature = spark.sql("""
SELECT userId,
       churn,  
       --Gender
       MAX(CASE WHEN gender = 'M' THEN 1 ELSE 0 END)  AS gender_male,
       MAX(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) AS gender_female,
       
       -- browser
       MAX(CASE WHEN get_browser(userAgent) = 'Chrome' THEN 1 ELSE 0 END)  AS chrome_user,
       MAX(CASE WHEN get_browser(userAgent) = 'Firefox' THEN 1 ELSE 0 END)  AS firefox_user,
       MAX(CASE WHEN get_browser(userAgent) = 'Safari' THEN 1 ELSE 0 END)  AS safari_user,
       MAX(CASE WHEN get_browser(userAgent) LIKE '%Microsoft%' THEN 1 ELSE 0 END)  AS ie_user,
       
       --platform
       --MAX(CASE WHEN get_platform(userAgent) = 'Linux' THEN 1 ELSE 0 END)  AS linux_user,
       --MAX(CASE WHEN get_platform(userAgent) = 'iOS' THEN 1 ELSE 0 END)  AS ios_user,
       --MAX(CASE WHEN get_platform(userAgent) = 'Mac OS' THEN 1 ELSE 0 END)  AS mac_user,
       --MAX(CASE WHEN get_platform(userAgent) = 'Windows' THEN 1 ELSE 0 END)  AS windows_user,
       
       
       
       
       -- Paid and free session
       COUNT(DISTINCT CASE WHEN level = 'paid' THEN sessionId ELSE 0 END) AS cnt_paid_session,
       COUNT(DISTINCT CASE WHEN level = 'free' THEN sessionId ELSE 0 END) AS cnt_free_session,
       
       -- Artist and song behavior
       --COUNT(DISTINCT artist) AS cnt_unique_artist,
       COUNT(DISTINCT song) AS cnt_unique_song,
       --AVG(length) AS average_song_length,
       SUM(CASE WHEN page = 'NextSong' THEN 1 ELSE 0 END) AS cnt_song_played,
    
       -- Other behavior
       COUNT(DISTINCT CASE WHEN page = 'NextSong' THEN from_unixtime(ts/1000, 'yyyy-MM-dd') ELSE 0 END) AS cnt_day_played,
       COUNT(DISTINCT sessionId) AS cnt_session,
       SUM(CASE WHEN page = 'Add Friend' THEN 1 ELSE 0 END) AS cnt_add_friend,
       SUM(CASE WHEN page = 'Thumbs Down' THEN 1 ELSE 0 END) AS cnt_thumb_down,
       SUM(CASE WHEN page = 'Thumbs Up' THEN 1 ELSE 0 END) AS cnt_thumbs_up,
       SUM(CASE WHEN page = 'Home' THEN 1 ELSE 0 END) AS cnt_home,
       --SUM(CASE WHEN page = 'Logout' THEN 1 ELSE 0 END) AS cnt_logout,
       --SUM(CASE WHEN page = 'Help' THEN 1 ELSE 0 END) AS cnt_help,
       --SUM(CASE WHEN page = 'Settings' THEN 1 ELSE 0 END) AS cnt_settings,
       --SUM(CASE WHEN page = 'Submit Downgrade' THEN 1 ELSE 0 END) AS cnt_submit_downgrade,
       --SUM(CASE WHEN page = 'Submit Upgrade' THEN 1 ELSE 0 END) AS cnt_submit_upgrade,
       --SUM(CASE WHEN page = 'Downgrade' THEN 1 ELSE 0 END) AS cnt_initiate_downgrade,
       --SUM(CASE WHEN page = 'Upgrade' THEN 1 ELSE 0 END) AS cnt_initiate_upgrade,
       SUM(CASE WHEN page = 'Error' THEN 1 ELSE 0 END) AS cnt_error,
       MAX(ts-registration)/(1000*3600*365*24) AS user_age
       
       --Combining session feature
       --MAX(sf.avg_hour_per_session)  AS avg_hour_per_session,
       --MAX(sf.avg_unique_song_per_session) AS avg_unique_song_per_session,
       --MAX(sf.avg_unique_artist_per_session) AS avg_unique_artist_per_session,
       --MAX(sf.avg_song_played_persession) AS avg_song_played_persession
    
FROM event_with_churn 
--LEFT JOIN session_feature sf USING (userId)
GROUP BY 1,2     
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
full_feature.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[userId: string, churn: int, gender_male: int, gender_female: int, chrome_user: int, firefox_user: int, safari_user: int, ie_user: int, cnt_paid_session: bigint, cnt_free_session: bigint, cnt_unique_song: bigint, cnt_song_played: bigint, cnt_day_played: bigint, cnt_session: bigint, cnt_add_friend: bigint, cnt_thumb_down: bigint, cnt_thumbs_up: bigint, cnt_home: bigint, cnt_error: bigint, user_age: double]

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

Only selected features are used in this notebook compare to smaller dataseat to overcome EMR performance.

In [26]:
features_col = ['gender_male','gender_female','chrome_user','firefox_user','safari_user',
               'ie_user',
                #'linux_user','ios_user','mac_user','windows_user','cnt_paid_session',
               'cnt_free_session',#'cnt_unique_artist',
               'cnt_unique_song',
                #'average_song_length',
               'cnt_song_played','cnt_day_played','cnt_session','cnt_add_friend','cnt_thumb_down',
               'cnt_thumbs_up','cnt_home',
                #'cnt_logout','cnt_help','cnt_settings','cnt_submit_downgrade',
                #'cnt_submit_upgrade','cnt_initiate_downgrade','cnt_initiate_upgrade',
               'cnt_error','user_age'
                #,'avg_hour_per_session',
                #'avg_unique_song_per_session','avg_unique_artist_per_session'
                ]
assembler = VectorAssembler(inputCols=features_col, outputCol="feature_vec")

df = assembler.setHandleInvalid("skip").transform(full_feature)
train, test = df.randomSplit([0.8, 0.2], seed=42)

indexer = StringIndexer(inputCol="churn", outputCol="label")
scaler = StandardScaler(inputCol="feature_vec", outputCol="scaled_features")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic Regression as Benchmark

In [27]:

lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0, featuresCol='scaled_features')
pipeline_logreg = Pipeline(stages=[indexer, scaler, lr])
paramGrid = ParamGridBuilder().addGrid(lr.regParam,[0.0, 0.1, 1]).build()

crossval_lr = CrossValidator(estimator=pipeline_logreg,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                          numFolds=3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
cvModel_lr = crossval_lr.fit(train)
print('F1 score for LogReg model in train :',  cvModel_lr.avgMetrics)
results_lr = cvModel_lr.transform(test)

In [29]:
print("Accuracy for Logistic Regression Model in test: ", results_lr.filter(results_lr.label == results_lr.prediction).count()/ results_lr.count())
evaluator = MulticlassClassificationEvaluator(metricName='f1')
score = evaluator.evaluate(results_lr)
print("F1 score for Logistic Regression model in test : ", score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy for Logistic Regression Model in test:  0.8212699822380106
F1 score for Logistic Regression model in test :  0.7803118352464993

In [30]:
feature_importances = cvModel_lr.bestModel.stages[-1].coefficients.values.tolist()

def plot_feature(feature_importances, features_col):
    feature_importance_df = pd.DataFrame({'coefficients': feature_importances, 'columns': features_col})
    graph = sns.barplot(x='coefficients', y='columns', data=feature_importance_df.sort_values('coefficients',ascending=False))
    graph.set_title('Feature coefficient plot')
    
    
plot_feature(feature_importances,features_col)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Gradient Boosting Model

In [31]:
gbt = GBTClassifier(labelCol="label", featuresCol="scaled_features", maxIter=10)
pipeline_gbt = Pipeline(stages=[indexer, scaler, gbt])
paramGrid = ParamGridBuilder().build()

crossval_gbt = CrossValidator(estimator=pipeline_gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                          numFolds=3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
cvModel_gbt = crossval_gbt.fit(train)
print('F1 score for GBT model in train :',  cvModel_gbt.avgMetrics)
results_gbt = cvModel_gbt.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-31:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2346



F1 score for GBT model in train : [0.8077054267756896]

In [33]:
print('F1 score for GBT model in train :',  cvModel_gbt.avgMetrics)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 score for GBT model in train : [0.8077054267756896]

In [34]:
print("Accuracy for GBT Model in test: ", results_gbt.filter(results_gbt.label == results_gbt.prediction).count()/ results_gbt.count())
evaluator = MulticlassClassificationEvaluator(metricName='f1')
score = evaluator.evaluate(results_gbt)
print("F1 score for GBT model in test : ", score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy for GBT Model in test:  0.8441385435168739
F1 score for GBT model in test :  0.828449317082134

In [35]:
feature_importances = cvModel_gbt.bestModel.stages[2].featureImportances.values.tolist()
gbt_col = [features_col[x] for x in cvModel_gbt.bestModel.stages[2].featureImportances.indices.tolist()]

plot_feature(feature_importances,gbt_col)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Naive Bayes 

In [37]:

nb = NaiveBayes(smoothing=1.0, modelType="multinomial",featuresCol='scaled_features')
pipeline = Pipeline(stages=[indexer, scaler, nb])
paramGrid = ParamGridBuilder().build()

crossval_nb = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                          numFolds=3)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
cvModel_nb = crossval_nb.fit(train)
print('F1 score for Naive bayes model in train :',  cvModel_nb.avgMetrics)
results_nb = cvModel_nb.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o6297.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 16 in stage 3184.0 failed 4 times, most recent failure: Lost task 16.3 in stage 3184.0 (TID 153850, ip-172-31-4-136.eu-west-1.compute.internal, executor 7): java.lang.IllegalArgumentException: requirement failed: Naive Bayes requires nonnegative feature values but found [0.0,2.0016086174146306,0.0,0.0,2.5713021052493215,0.0,0.41638696427841526,0.05860759316046543,0.047807726206852856,0.3744267831483405,0.2307709434337849,0.04908113282955325,0.0,0.06171763548007865,0.022030137244053144,0.0,-1.8829414639316775E-4].
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.classification.NaiveBayes$.requireNonnegativeValues(NaiveBayes.scala:235)
	at org.apache.spark.ml.classification.NaiveBayes$$anonfun$trainWithLabelCheck$1$$anonfun$4.apply(NaiveBayes.scala:144)
	at org.apache.spark.ml.classification.NaiveBayes$$anonfun$trainWithLabelChe

In [39]:
print("Accuracy for NB Model in test: ", results_nb.filter(results_nb.label == results_nb.prediction).count()/ results_nb.count())
evaluator = MulticlassClassificationEvaluator(metricName='f1')
score = evaluator.evaluate(results_nb)
print("F1 score for NB model in test : ", score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'results_nb' is not defined
Traceback (most recent call last):
NameError: name 'results_nb' is not defined



Random forrest

In [42]:

rf = RandomForestClassifier(labelCol="label", featuresCol="scaled_features", numTrees=10)
pipeline = Pipeline(stages=[indexer, scaler, rf])
paramGrid = ParamGridBuilder().build()

crossval_rf = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                          numFolds=3)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
cvModel_rf = crossval_rf.fit(train)
print('F1 score for Random Forrest model in train :',  cvModel_rf.avgMetrics)
results_rf = cvModel_rf.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-42:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 3290



F1 score for Random Forrest model in train : [0.8023975974731592]

In [44]:
print("Accuracy for Random Forrest Model in test: ", results_rf.filter(results_rf.label == results_rf.prediction).count()/ results_rf.count())
evaluator = MulticlassClassificationEvaluator(metricName='f1')
score = evaluator.evaluate(results_rf)
print("F1 score for Random Forrest model in test : ", score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-43:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 3493



Accuracy for Random Forrest Model in test:  0.8368117229129662
F1 score for Random Forrest model in test :  0.8184387982397876

In full dataset, Random forrest has the highest F1 score compare to others

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.